## CSV Preprocess

In [15]:
"""
! conda activate Seol
! pip install geopandas
! pip install shapely
"""

import pandas as pd
import os 
from geopandas import GeoDataFrame
import pickle
from shapely.geometry import Point

cur_path = os.getcwd() # 경로 설정

# subway_line= pd.read_csv('서울시 역사마스터 정보.csv', engine='python', encoding='utf-8')
subway_line= pd.read_csv('서울시 역사마스터 정보 최종.csv', engine='python', encoding='utf-8')

In [16]:
subway_line.head()

,역사_ID,역사명,호선,위도,경도
0,150,서울역,1호선,37.554648,126.972559
1,151,시청,1호선,37.564718,126.977108
2,152,종각,1호선,37.570161,126.982923
3,153,종로3가,1호선,37.571607,126.991806
4,154,종로5가,1호선,37.570926,127.001849


In [23]:
subway_line.describe

<bound method NDFrame.describe of      역사_ID     역사명     호선         위도          경도
0      150     서울역    1호선  37.554648  126.972559
1      151      시청    1호선  37.564718  126.977108
2      152      종각    1호선  37.570161  126.982923
3      153    종로3가    1호선  37.571607  126.991806
4      154    종로5가    1호선  37.570926  127.001849
..     ...     ...    ...        ...         ...
768   3223   모래내시장  인천2호선  37.455830  126.719298
769   3224      만수  인천2호선  37.454911  126.732094
770   3225    남동구청  인천2호선  37.448161  126.736939
771   3226   인천대공원  인천2호선  37.448769  126.752618
772   3227  운연(서창)  인천2호선  37.440127  126.759970

[773 rows x 5 columns]>

In [18]:
subway_line["역사정보"].head()

0    150
1    151
2    152
3    153
4    154
Name: 역사_ID, dtype: int64

In [19]:
list(subway_line.loc[(subway_line["역사명"] == '하계'), ["역사명", "위도", "경도"]].itertuples(name=None))

[(348, '하계', 37.636352, 127.06799)]

In [20]:
subway_line.loc[(subway_line["역사명"] == '충무로'), :]

,역사_ID,역사명,호선,위도,경도
161,321,충무로,3호선,37.561430,126.994072
211,423,충무로,4호선,37.561207,126.994080


## Excel Preprocessing

In [30]:
! conda activate Seol
! pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [50]:
from openpyxl import load_workbook

load_wb = load_workbook("./서울 지하철 역간 거리 정보.xlsx", data_only=True)

In [52]:
seoul_underground = ['1호선', '2호선', '3호선', 
                     '4호선', '5호선', '6호선', '7호선', '8호선', '9호선']

In [51]:
load_ws = load_wb['1호선']
# print(load_ws.cell(3,2).value)

In [47]:
for line in seoul_underground:
    load_ws = load_wb[line]

number line name distance 
1 1호선 소요산 0 
2 1호선 동두천 2.5 
3 1호선 보산 1.6 
4 1호선 동두천중앙 1.4 
5 1호선 지행 1 
6 1호선 덕정 5.6 
7 1호선 덕계 2.9 
8 1호선 양주 5.3 


In [ ]:
get_cells = load_ws['A1':'D9']
    for row in get_cells:
        for cell in row:
            print(cell.value, end=' ')
        print("")

In [2]:
def find_all_paths(graph, start, end, path=[]):
        path = path + [start]
        if start == end:
            return [path]
        if start not in graph:
            return []
        paths = []
        for node in graph[start]:
            if node not in path:
                newpaths = find_all_paths(graph, node, end, path)
                for newpath in newpaths:
                    paths.append(newpath)
        return paths       

def min_path(graph, start, end):
    paths=find_all_paths(graph,start,end)
    mt=10**99
    mpath=[]
    print('\tAll paths:',paths)
    for path in paths:
        t=sum(graph[i][j] for i,j in zip(path,path[1::]))
        print('\t\tevaluating:',path, t)
        if t<mt: 
            mt=t
            mpath=path

    e1=' '.join('{}->{}:{}'.format(i,j,graph[i][j]) for i,j in zip(mpath,mpath[1::]))
    e2=str(sum(graph[i][j] for i,j in zip(mpath,mpath[1::])))
    print('Best path: '+e1+'   Total: '+e2+'\n')

if __name__ == "__main__":
    graph = {'A': {'B':5, 'C':4},
             'B': {'C':3, 'D':10},
             'C': {'D':12},
             'D': {'C':5, 'E':9},
             'E': {'F':8},
             'F': {'C':7}}
    min_path(graph,'A','E')
    min_path(graph,'A','D')
    min_path(graph,'A','F')

	All paths: [['A', 'B', 'C', 'D', 'E'], ['A', 'B', 'D', 'E'], ['A', 'C', 'D', 'E']]
		evaluating: ['A', 'B', 'C', 'D', 'E'] 29
		evaluating: ['A', 'B', 'D', 'E'] 24
		evaluating: ['A', 'C', 'D', 'E'] 25
Best path: A->B:5 B->D:10 D->E:9   Total: 24

	All paths: [['A', 'B', 'C', 'D'], ['A', 'B', 'D'], ['A', 'C', 'D']]
		evaluating: ['A', 'B', 'C', 'D'] 20
		evaluating: ['A', 'B', 'D'] 15
		evaluating: ['A', 'C', 'D'] 16
Best path: A->B:5 B->D:10   Total: 15

	All paths: [['A', 'B', 'C', 'D', 'E', 'F'], ['A', 'B', 'D', 'E', 'F'], ['A', 'C', 'D', 'E', 'F']]
		evaluating: ['A', 'B', 'C', 'D', 'E', 'F'] 37
		evaluating: ['A', 'B', 'D', 'E', 'F'] 32
		evaluating: ['A', 'C', 'D', 'E', 'F'] 33
Best path: A->B:5 B->D:10 D->E:9 E->F:8   Total: 32



## CSV 파일 기반 노선도 그래프 구축

: 상윤이가 하기로 함 (Firebase import/export w/ csv)

In [ ]:
! conda activate Seol
! pip install firebase_admin

In [38]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import json

cred = credentials.Certificate\
    ("freshroute-b533b-firebase-adminsdk-dr0b0-5c0276c696.json")

firebase_admin.initialize_app(cred, {
  'projectId': 'freshroute-b533b',
})

db = firestore.client()
doc_ref = db.collection("Subway").document("1002")

doc = doc_ref.get()
if doc.exists:
    print(f"Document data: {doc.to_dict()}")
else:
    print("No such document!")

ModuleNotFoundError: No module named 'firebase_admin'

# From Local json

In [2]:
import json
import pprint
import copy

with open("./data/bus.json", "r") as bus:
    map_bus = json.load(bus)
with open("./data/subway.json", "r") as subway:
    map_subway = json.load(subway)
with open("./data/subway_transfer.json", "r") as transfer:
    map_trans = json.load(transfer)

In [3]:
for tmp in map_subway:
    if map_subway[tmp]['name'] == '사당':
        print(tmp)

226
433


In [4]:
pp = pprint.PrettyPrinter(depth=4)
# pp.pprint(map_subway)
# pp.pprint(map_trans)
pp.pprint(map_subway['226'])
pp.pprint(map_subway['433'])
print("\n")
pp.pprint(map_trans['226'])
pp.pprint(map_trans['433'])
print("\n")
pp.pprint(map_bus['117000050'])

{'adj': {'0': {'dist': 1.6, 'id': '225', 'line': '2호선', 'name': '방배'},
         '1': {'dist': 1.7, 'id': '227', 'line': '2호선', 'name': '낙성대'}},
 'id': '226',
 'latitude': 37.476538,
 'line': '2호선',
 'longitude': 126.981544,
 'name': '사당'}
{'adj': {'0': {'dist': 1.6, 'id': '434', 'line': '4호선', 'name': '남태령'},
         '1': {'dist': 1.1, 'id': '432', 'line': '4호선', 'name': '이수'}},
 'id': '433',
 'latitude': 37.476955,
 'line': '4호선',
 'longitude': 126.981651,
 'name': '사당'}


{'id': '226',
 'line': '2호선',
 'name': '사당',
 'transferInfo': {'dist': 74, 'line': '4호선', 'name': '사당', 'time': '1분2초'}}
{'id': '433',
 'line': '4호선',
 'name': '사당',
 'transferInfo': {'dist': 74, 'line': '2호선', 'name': '사당', 'time': '1분2초'}}


{'adj': {'0': {'ARS_ID': '21172',
               'NODE_ID': 120000070,
               'ROUTE_ID': 100100521,
               'X좌표': 126.922015,
               'Y좌표': 37.458697,
               '노선명': '6515',
               '순번': 53,
               '정류소명': '산복터널.관악산휴먼시아2단지'},
  

In [5]:
map_bus_tmp = copy.deepcopy(map_bus)

# Redundant values
# 
# ARS_ID, mobile_number ()
# ID, NODE_ID 동일 ()
# ROUTE_ID, 노선명 1대1 대응

# pp.pprint(map_bus)


for tmp in map_bus_tmp:
    map_bus_tmp[tmp]['node_id'] = map_bus_tmp[tmp].pop('id')
    map_bus_tmp[tmp]['ars_id'] = map_bus_tmp[tmp].pop('mobile_number')
    try:
        for tmp2 in map_bus_tmp[tmp]['adj']:
            map_bus_tmp[tmp]['adj'][tmp2]['node_id'] = map_bus_tmp[tmp]['adj'][tmp2].pop('NODE_ID')
            map_bus_tmp[tmp]['adj'][tmp2]['latitude'] = map_bus_tmp[tmp]['adj'][tmp2].pop('Y좌표')
            map_bus_tmp[tmp]['adj'][tmp2]['longitude'] = map_bus_tmp[tmp]['adj'][tmp2].pop('X좌표')
            map_bus_tmp[tmp]['adj'][tmp2]['station_name'] = map_bus_tmp[tmp]['adj'][tmp2].pop('정류소명')
            map_bus_tmp[tmp]['adj'][tmp2]['bus_number'] = map_bus_tmp[tmp]['adj'][tmp2].pop('노선명')
            map_bus_tmp[tmp]['adj'][tmp2]['ars_id'] = map_bus_tmp[tmp]['adj'][tmp2].pop('ARS_ID')
            map_bus_tmp[tmp]['adj'][tmp2]['route_id'] = map_bus_tmp[tmp]['adj'][tmp2].pop('ROUTE_ID')
    except:
        print("",end='')

## Node class 정의

In [25]:
class Node:
    def __init__(self, id=None, name=None, lat=None, long=None, adj=None):
        self.id = id
        self.name = name
        
        self.lat = lat
        self.long = long
        
        self.adj = []

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.lat == other.lat and self.long == other.long

## Heuristics

In [26]:
def heuristic(node, goal, D=1, D2=2 ** 0.5):
    dx = abs(node.lat - goal.lat)
    dy = abs(node.long - goal.long)
    return D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)

 ## A-star class

In [54]:
# start = list(subway_line.loc[(subway_line["역사명"] == '하계'), ["역사명", "위도", "경도"]].itertuples(name=None))
# end = list(subway_line.loc[(subway_line["역사명"] == '충무로'), ["역사명", "위도", "경도"]].itertuples(name=None))

start = list(subway_line.loc[(subway_line["역사명"] == '하계'), ["역사명", "위도", "경도"]].to_records()[0])
end = list(subway_line.loc[(subway_line["역사명"] == '충무로'), ["역사명", "위도", "경도"]].to_records()[0])

startNode = Node(*start)
endNode = Node(*end)
openList = []
closedList = []

print(startNode.id, endNode.id)
openList.append(startNode)
# print(openList)

348 161


In [55]:
while openList:
    
    currentNode = openList[0]
    currentIdx = 0
    
    # print(currentNode, currentIdx)

    for index, item in enumerate(openList):
        if item.f < currentNode.f:
            currentNode = item
            currentIdx = index

    openList.pop(currentIdx)
    closedList.append(currentNode)

    if currentNode == endNode:
        path = []
        current = currentNode
        while current is not None:
            path.append(current.lat, current.long)
            current = current.parent
        print(path[::-1])

    children = []

    for newID in currentNode.adj:

        new_node = Node(*(list(subway_line.loc[(subway_line["역사_ID"] == newID), ["역사명", "위도", "경도"]].to_records()[0])))
        children.append(new_node)

    for child in children:

        if child in closedList:
            continue

        # f, g, h값 업데이트
        child.g = currentNode.g + 1
        child.h = ((child.lat - endNode.lat) **
                   2) + ((child.lat - endNode.lat) ** 2)
        # child.h = heuristic(child, endNode) 다른 휴리스틱
        # print("position:", child.position) 거리 추정 값 보기
        # print("from child to goal:", child.h)

        child.f = child.g + child.h

        if len([openNode for openNode in openList
                if child == openNode and child.g > openNode.g]) > 0:
            continue

        openList.append(child)

<__main__.Node object at 0x00000206821FDDF0> 0


## Backbone code

In [1]:
class Node:
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

def heuristic(node, goal, D=1, D2=2 ** 0.5):
    dx = abs(node.position[0] - goal.position[0])
    dy = abs(node.position[1] - goal.position[1])
    return D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)


def aStar(maze, start, end):
    
    startNode = Node(None, start)
    endNode = Node(None, end)

    openList = []
    closedList = []

    openList.append(startNode)

    while openList:

        currentNode = openList[0]
        currentIdx = 0

        for index, item in enumerate(openList):
            if item.f < currentNode.f:
                currentNode = item
                currentIdx = index

        openList.pop(currentIdx)
        closedList.append(currentNode)

        if currentNode == endNode:
            path = []
            current = currentNode
            while current is not None:
                # x, y = current.position
                path.append(current.position)
                current = current.parent
            return path[::-1]  # reverse

        children = []
        
        # 인접한 xy좌표 전부
        for newPosition in [(0, -1), (0, 1), (-1, 0), (1, 0), (-1, -1), (-1, 1), (1, -1), (1, 1)]:

            # 노드 위치 업데이트
            nodePosition = (
                currentNode.position[0] + newPosition[0],  # X
                currentNode.position[1] + newPosition[1])  # Y
                
            # 미로 maze index 범위 안에 있어야함
            within_range_criteria = [
                nodePosition[0] > (len(maze) - 1),
                nodePosition[0] < 0,
                nodePosition[1] > (len(maze[len(maze) - 1]) - 1),
                nodePosition[1] < 0,
            ]

            if any(within_range_criteria):  # 하나라도 true면 범위 밖임
                continue

            # 장애물이 있으면 다른 위치 불러오기
            if maze[nodePosition[0]][nodePosition[1]] != 0:
                continue

            new_node = Node(currentNode, nodePosition)
            children.append(new_node)

        # 자식들 모두 loop
        for child in children:

            # 자식이 closedList에 있으면 continue
            if child in closedList:
                continue

            # f, g, h값 업데이트
            child.g = currentNode.g + 1
            child.h = ((child.position[0] - endNode.position[0]) **
                       2) + ((child.position[1] - endNode.position[1]) ** 2)
            # child.h = heuristic(child, endNode) 다른 휴리스틱
            # print("position:", child.position) 거리 추정 값 보기
            # print("from child to goal:", child.h)
            
            child.f = child.g + child.h

            if len([openNode for openNode in openList
                    if child == openNode and child.g > openNode.g]) > 0:
                continue
                    
            openList.append(child)


def main():
    # 1은 장애물
    maze = [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

    start = (0, 0)
    end = (7, 6)

    path = aStar(maze, start, end)
    print(path)


if __name__ == '__main__':
    main()



[(0, 0), (1, 1), (2, 2), (3, 3), (4, 3), (5, 4), (6, 5), (7, 6)]


## Class